In [2]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib as plt
import pandas as pd

In [3]:
df = pd.read_csv("../data/rentsmart.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64099 entries, 0 to 64098
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            64099 non-null  object 
 1   violation_type  64099 non-null  object 
 2   description     64099 non-null  object 
 3   address         64099 non-null  object 
 4   neighborhood    64099 non-null  object 
 5   zip_code        64099 non-null  int64  
 6   parcel          64099 non-null  int64  
 7   owner           64099 non-null  object 
 8   year_built      64099 non-null  int64  
 9   year_remodeled  64099 non-null  int64  
 10  property_type   64099 non-null  object 
 11  latitude        64099 non-null  float64
 12  longitude       64099 non-null  float64
 13  year            64099 non-null  int64  
dtypes: float64(2), int64(5), object(7)
memory usage: 6.8+ MB


In [5]:
district_shapefile = gpd.read_file("../data/City-Council-District")

In [6]:
count = 0
is_D7_addresses = []

for row in df.itertuples(index=True, name="Row"):
    address_point = Point(row.longitude, row.latitude)
    address_gdf = gpd.GeoDataFrame(geometry=[address_point], crs="EPSG:4326")
    address_gdf = address_gdf.to_crs(district_shapefile.crs)
    result = gpd.sjoin(address_gdf, district_shapefile, how="left", predicate="intersects")

    if result['DISTRICT'].values[0] == 7:
        count += 1
        is_D7_addresses.append(True)
    else:
        is_D7_addresses.append(False)

In [9]:
print(count)

21875


In [10]:
df['is_d7'] = is_D7_addresses
df

,date,violation_type,description,address,neighborhood,zip_code,parcel,owner,year_built,year_remodeled,property_type,latitude,longitude,year,is_d7
0,2024-10-02,Enforcement Violations,Improper storage trash: res,"1081-1083 Dorchester Ave, 02125",Dorchester,2125,1302139000,DORCHESTER FIRST REALTY TRUST,1905,2014,Residential 3-family,42.313779,-71.057314,2024,False
1,2024-10-02,Enforcement Violations,Improper storage trash: res,"6 Dawes Ter, 02125",Dorchester,2125,703977000,MADEK CHARLES V,1905,1970,Residential 3-family,42.318930,-71.060761,2024,False
2,2024-10-02,Enforcement Violations,Overfilling of barrel/dumpster,"7 Bakersfield St, 02125",Dorchester,2125,1301796000,CESCO LLC MASS LLC,1936,1981,Residential 7 or more units,42.317846,-71.060866,2024,False
3,2024-10-01,Enforcement Violations,Overgrown Weeds On Property,"27 Woodbine St, 02119",Roxbury,2119,1200848000,WHITE GILBERT G III,1890,2010,Residential 1-family,42.318880,-71.079861,2024,True
4,2024-10-01,Enforcement Violations,illegal parking prop owner 1,"2 Humboldt Ave, 02119",Roxbury,2119,1201598250,TWO HUMBOLDT AV CONDO TR,1999,1999,Condominium Main*,42.320570,-71.085741,2024,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64094,2019-10-29,Housing Complaints,Unsatisfactory Living Conditions,"45 Dwight St #A, 02118",Roxbury,2118,305741000,HISTORIC SOUTH END LP MASS LP,1900,1990,Residential 7 or more units,42.344531,-71.069138,2019,False
64095,2019-10-29,Housing Complaints,Plumbing,"610 Tremont St, 02118",Roxbury,2118,900091000,PHIL C BRADLEY HOUSING,1920,1998,Mixed Use (Res. and Comm.),42.342540,-71.073708,2019,False
64096,2019-10-29,Housing Complaints,Unsatisfactory Living Conditions,"56 Bowdoin Ave #4, 02121",Dorchester,2121,1401292000,ALLEN MARY ANN,1950,2004,Residential 7 or more units,42.301370,-71.074491,2019,False
64097,2019-10-29,Housing Complaints,Rental Unit Delivery Conditions,"5 S Charlame Ct, 02119",Roxbury,2119,1201125000,CHARLAME PARK HOMES I COOP,1964,1990,Residential 7 or more units,42.319490,-71.085619,2019,True


In [12]:
df.to_csv("../data/d7-rentsmart-new.csv", index=False)